# Исследование надежности заемщиков.

**Цель проекта: изучить зависимость семейного положения и количество детей клиента на факт погашения кредита в срок.**

Проверить следующие теории:
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

Входные данные от банка — статистика о платёжеспособности клиентов.


План работы:
* изучить данные
* предобработка
* категоризация данных
* проверка теорий

## Шаг 1. Обзор данных

### Загружаем данные

Импортируем библиотеку Pandas
Загружаем данные

In [162]:
import pandas as pd

In [163]:
data = pd.read_csv('/datasets/data.csv')

### Изучаем основные сведения о данных.

In [164]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В DataFrame 12 колонок и 21525 строк.

Просматриваем данные:

In [165]:
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В колонке `days_employed` имеются отрицательные значения и имеет тип `float64`. 
Причины указания отрицательных значений нам не известны, однако данный факт может негативно отразится на общем анализе.

Считаем количество пропусков

In [166]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


В колонках `days_employed` (общий трудовой стаж в днях) и `total_income` (ежемесячный доход) имеются пропуски.

In [167]:
print('Доля пропущенных значений составила:', int(2174 / 21525 * 100), '%')

Доля пропущенных значений составила: 10 %


Удаление 10% значений негативно отразится на общих результатах анализа, соответственно удалять пропущенные значения нельзя.
Необходимо заполнить пропуски 

Столбец `days_employed` (общий трудовой стаж) в данном анализе не используется, в связи с чем проверять его значения и исправлять данные не имеет смысла. Исправить отрицательные значения можно с помощью метода `abs()`, однако пересчитать стаж и найти другие аномалии трудозатратно. Возможно в некоторых строках стаж указан в часах, пример:

In [168]:
print('Значение стажа в 5 строке', data['days_employed'][4], 'составляет', int(data['days_employed'][4] / 365), 'года')

Значение стажа в 5 строке 340266.07204682194 составляет 932 года


В данном случае необходимо обратиться к источнику данных.

Возможные причины аномалий:
- неккоректное заполнение (опечатка)
- взятие данных из нескольких источников с разным форматом записи даты
- некорректное написание формулы расчета дней
- взятие различных дат при расчете дней
- и д.р.

Заменяем пустые значения в столбце `days_employed` на `unknown`

In [169]:
data['days_employed'] = data['days_employed'].fillna('unknown')

## Предобработка данных

### Шаг 2.1 Заполнение пропусков

Заполняем пропуски в столбце `total_income` значением медианы.

Расчитываем медиану для солбца `total_income`:

In [170]:
total_income_avg = data['total_income'].median()
print(f'Медиана для солбца total_income равна {total_income_avg}')

Медиана для солбца total_income равна 145017.93753253992


Заменяем пропуски на значения медианы:

In [171]:
data['total_income'] = data['total_income'].fillna(total_income_avg)

Проверяем:

In [172]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Все пропуски в столбце `total_income` исправлены

### Шаг 2.2 Проверка данных на аномалии и исправления.

Проверяем данные в столбце `children`:

In [173]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Отрицаельное значение в данном столбце будем считать как опечатку, поэтому заменим на абсолютное значение:

In [174]:
data['children'] = data['children'].replace(-1, 1)

В датасете имеются сведения о 76 семьях с 20 детьми. Можно предположить, что это опечатка и детей все же 2. Заменим значения.

In [175]:
data['children'] = data['children'].replace(20, 2)

Проверяем:

In [176]:
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Проверяем данные в столбце `total_income` на наличие отрицательных и нулевых значений:

In [177]:
print('Количество отрицательных и нулевых значений:', data[data['total_income'] <= 0]['total_income'].count())

Количество отрицательных и нулевых значений: 0


### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце total_income на целочисленный с помощью функции `astype('int')`, так как при проверке гипотез количественные данные столбца будут использованы в качестве категориальных.
В данном случае возможно использовать конструкцию `try-except` на случай наличия ошибок заполнения. Мы заранее проверили тип данных, но данный код будет универсальным.  

In [178]:
try:
    data['total_income'] = data['total_income'].astype('int')
except:
    print('Ошибка в типе данных')

Проверяем:

In [179]:
display(data.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.67,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.8,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.42,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.75,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.4. Удаление дубликатов.

Удаляем явные дубликаты с обновлением индексов:

In [180]:
print('Количество дубликатов до удаления:', data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов после удаления:', data.duplicated().sum())

Количество дубликатов до удаления: 54
Количество дубликатов после удаления: 0


Проверяем значения столбца `family_status`:

In [181]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

Аномалии отсутствуют.

Проверяем значения столбца `education`:

In [182]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Некоторые значения в столбце написаны строчными буквами. Меняем на прописные и проверяем:

In [183]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Повторно проверяем дубликаты:

In [184]:
print('Количество дубликатов до удаления:', data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов после удаления:', data.duplicated().sum())

Количество дубликатов до удаления: 17
Количество дубликатов после удаления: 0


### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

В датафрейме имются столбцы, которые фактически дублируют данные и загружают объем обрабатываемых данных
* `education` и `education_id`
* `family_status`  и `family_status_id`
* `purpose` и `purpose_group`

Данные столбцы можно выделить в отдельные словари и отавить в датафрейме только идентифаторы.

Создаем датафрейм-словарь `education_info` и удаляем дубликаты:

In [185]:
education_info = data[['education_id', 'education']]
education_info = education_info.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов после удаления:', education_info.duplicated().sum())

Количество дубликатов после удаления: 0


In [186]:
display(education_info)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Создаем датафрейм-словарь `family_status_info` и удаляем дубликаты:

In [187]:
family_status_info = data[['family_status_id', 'family_status']]
family_status_info = family_status_info.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов после удаления:', family_status_info.duplicated().sum())

Количество дубликатов после удаления: 0


In [188]:
display(family_status_info)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Удаляем из датафрейма столбцы `education` и `family_status`. Необходимые сведения из `purpose` мы уже отсортировали в отдельный столбец, необходимость в нем отсутствует.

In [189]:
data = data.drop(['education', 'family_status'], axis=1)

In [190]:
display(data.head(5))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.67,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.8,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.42,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.75,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных в столбце `total_income` присваеваем категорию доходов:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.


Создаем функцию для присвоения категории:

In [191]:
def total_income_group (total):
    """возвращает категорию дохода в зависимости от суммы"""
    if total <= 30000:
        return 'E'
    elif total <= 50000:
        return 'D'
    elif total <= 200000:
        return 'C'
    elif total <= 1000000:
        return 'B'
    else:
        return 'A'

Создаем столбец `total_income_category` с указанием категории дохода:

In [192]:
data['total_income_category'] = data['total_income'].apply(total_income_group)

### Шаг 2.7. Категоризация целей кредита.

В нашей работе есть гепотезы влияния цели кредита на сроки его возврата. В связи с чем необходимо провести категоризацию данных их столбца `purpose`.

Проверяем значения столбца `purpose`:

In [193]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создаем функцию для разделения вариантов на группы `недвижимость`, `автомобиль`, `свадьба`, `образование` с помощью фильтра на совпадение части слова.

In [194]:
def purpose_group (purpose):
    """возвращает группу цели кридита"""
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'недвижим' in purpose:
        return 'операции с недвижимостью'
    elif 'автом' in purpose:
        return 'операции с автомобилем'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образов' in purpose:
        return 'получение образования'

Создаем столбец с новой классификацией цели кридита.

In [195]:
data['purpose_group'] = data['purpose'].apply(purpose_group)

Проверяем:

In [196]:
display(data.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_group
0,1,-8437.67,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,-4024.8,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,-5623.42,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,-4124.75,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,-926.186,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,-2879.2,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,-152.78,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,-6929.87,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,-2188.76,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


Вывод: проведена работа по предобработке данных, в том числе:
- заполнены пропуски значений на медиану;
- проведена работа по проверке на аномальные данные, исправили опечатки;
- иправлен тип данных;
- удалены явные и скрытые дубликаты;
- сформированны словари;
- проведена категоризация дохода и целей кредита.

## Ответы на вопросы.

### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Создаем сводную таблицу с расчетом количества кредитов в разрезе наличия детей и наличия/отсутствия задолженности.

In [197]:
children_theory = data.pivot_table(index='children', columns='debt', values='income_type', aggfunc='count')

In [198]:
display(children_theory)

debt,0,1
children,,
0,13028.0,1063.0
1,4410.0,445.0
2,1926.0,202.0
3,303.0,27.0
4,37.0,4.0
5,9.0,NaN


Добавляем столбец `debt_avg` с расчетом соотношения количества просроченных кредитов к общему количеству в процентах, по каждой группе. И столбец `credit_children` с расчетом доли данной катогории кредитов к общему объму.

In [199]:
children_theory['debt_avg'] = children_theory[1] / (children_theory[1] + children_theory[0]) * 100
children_theory['credit_children'] = (children_theory[1] + children_theory[0]) / len(data['children']) * 100

In [200]:
display(children_theory)

debt,0,1,debt_avg,credit_children
children,,,,
0,13028.0,1063.0,7.543822,65.680060
1,4410.0,445.0,9.165808,22.629813
2,1926.0,202.0,9.492481,9.918896
3,303.0,27.0,8.181818,1.538175
4,37.0,4.0,9.756098,0.191107
5,9.0,NaN,NaN,NaN


### Вывод 1:

Из расчитанных данных можно сделать следующие выводы:
* бездетные берут кредиты чаще (65%), доля просроченных кредитов самая низкая 7,5%
* наличие от 1 до 4 детей увеличивает риск невозврата до 8-9,7%
* доля семей с 5 детьми незначительна, однако задолженность у них отсутствует.

### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создаем сводную таблицу с расчетом количества кредитов в разрезе семейного положения и наличия/отсутствия задолженности.

In [201]:
family_status_teory = data.pivot_table(index='family_status_id', columns='debt', values='income_type', aggfunc='count')

In [202]:
display(family_status_teory)

debt,0,1
family_status_id,,
0,11408,931
1,3763,388
2,896,63
3,1110,85
4,2536,274


Добавляем столбец `debt_avg` с расчетом соотношения количества просроченных кредитов к общему количеству в процентах, по каждой группе. И столбец `credit_family` с расчетом доли данной катогории кредитов к общему объму.

In [203]:
family_status_teory['debt_avg'] = family_status_teory[1] / (family_status_teory[1] + family_status_teory[0]) * 100
family_status_teory['credit_family'] = (family_status_teory[1] + family_status_teory[0]) / len(data['family_status_id']) * 100

In [204]:
display(family_status_teory)

debt,0,1,debt_avg,credit_family
family_status_id,,,,
0,11408,931,7.545182,57.513750
1,3763,388,9.347145,19.348373
2,896,63,6.569343,4.470029
3,1110,85,7.112971,5.570057
4,2536,274,9.750890,13.097791


Из словаря добавляем столбец с обозначением кодов

In [205]:
family_status_teory.merge(family_status_info, on='family_status_id', how='left')

,family_status_id,0,1,debt_avg,credit_family,family_status
0,0,11408,931,7.545182,57.513750,женат / замужем
1,1,3763,388,9.347145,19.348373,гражданский брак
2,2,896,63,6.569343,4.470029,вдовец / вдова
3,3,1110,85,7.112971,5.570057,в разводе
4,4,2536,274,9.750890,13.097791,Не женат / не замужем


### Вывод 2:

Из расчитанных данных можно сделать следующие выводы:
* наибольший объем кредитов приходится на семейные пары, более 57%. Риск невозврата 7,54%, что немного больше чем у разведенных - 7,11%;
* минимальный риск невозврата кредитов приходится на вдовствующих заемщиков - 6,57%. Однако данная категория самая малочисленная, всего 4,5% всех заемщиков;
* наибольший риск невозврата у людей находящихся в гражданском браке и холостых - 9,32% и 9,75% соответственно. 

### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создаем сводную таблицу с расчетом количества кредитов в разрезе уровня дохода и наличия/отсутствия задолженности.

In [206]:
total_income_teory = data.pivot_table(index='total_income_category', columns='debt', values='income_type', aggfunc='count')

In [207]:
display(total_income_teory)

debt,0,1
total_income_category,,
A,23,2
B,4685,356
C,14656,1360
D,329,21
E,20,2


Добавляем столбец `debt_avg` с расчетом соотношения количества просроченных кредитов к общему количеству в процентах, по каждой группе. И столбец `credit_total` с расчетом доли данной катогории кредитов к общему объму.

In [208]:
total_income_teory['debt_avg'] = total_income_teory[1] / (total_income_teory[1] + total_income_teory[0]) * 100
total_income_teory['credit_total'] = (total_income_teory[1] + total_income_teory[0]) / len(data['total_income_category']) * 100

In [209]:
display(total_income_teory)

debt,0,1,debt_avg,credit_total
total_income_category,,,,
A,23,2,8.000000,0.116528
B,4685,356,7.062091,23.496784
C,14656,1360,8.491508,74.652745
D,329,21,6.000000,1.631397
E,20,2,9.090909,0.102545


Категории:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

### Вывод 3:

Из расчитанных данных можно сделать следующие выводы:
* минимальный риск невозврата кредита (6%) приходится на граждан, уровень дохода которых от 30 до 50 тысяч;
* максимальный риск (9,09%) у граждан с уровнем дохода до 30 тысяч;
* граждане с уровнем дохода до 200 тысяч, а это 74% заемщиков, не возвращают кредит в 8,48% случаев;
* самые обеспеченные граждане, с уровнем дохода более 1 млн, так же не являются допропорядочными - риск 8,00%

Теория подтверждена.

### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

Создаем сводную таблицу с расчетом количества кредитов в разрезе целей заема и наличия/отсутствия задолженности.

In [210]:
purpose_teory = data.pivot_table(index='purpose_group', columns='debt', values='income_type', aggfunc='count')

In [211]:
display(purpose_teory)

debt,0,1
purpose_group,,
операции с автомобилем,3903,403
операции с недвижимостью,10029,782
получение образования,3643,370
проведение свадьбы,2138,186


Добавляем столбец `debt_avg` с расчетом соотношения количества просроченных кредитов к общему количеству в процентах, по каждой группе. И столбец `purpose_family` с расчетом доли данной катогории кредитов к общему объму.

In [212]:
purpose_teory['debt_avg'] = purpose_teory[1] / (purpose_teory[1] + purpose_teory[0]) * 100
purpose_teory['credit_purpose'] = (purpose_teory[1] + purpose_teory[0]) / len(data['family_status_id']) * 100

In [213]:
display(purpose_teory)

debt,0,1,debt_avg,credit_purpose
purpose_group,,,,
операции с автомобилем,3903,403,9.359034,20.070849
операции с недвижимостью,10029,782,7.233373,50.391535
получение образования,3643,370,9.220035,18.705137
проведение свадьбы,2138,186,8.003442,10.832479


### Вывод 4:

Из расчитанных данных можно сделать следующие выводы:
* наибольший объем кредитов приходится на операции с недвижимостью, более 50%. Риск невозврата в данной категории минимальный 7,23%;
* для приобретения автомобиля и получения образования кредит берут в 20% и 18% случаем соответственно, при этом не возвращают кредиты 9% заемщиков;
* меньше всего заем берут для проведения свадьбы (10%), однако возвращают охотнее - риск 7,97%.

## Общий вывод:

По результатам анализа можно сделать следующие выводы:
* охотнее кредит берут:
    - семейные пары
    - без детей
    - на операции с недвижимостью
    - с уровнем дохода от 50 до 200 тысяч
* самыми дисциплинированными являются:
    - вдовцы
    - с 5 детьми, либо бездетные
    - с уровнем дохода от 30 до 50 тысяч
    - желающие улучшить свои жилищные условия
* чаще всего отказ в выдаче заема услышат:
    - находящихся в гражданском браке и холостые
    - с 2-4 детьми на иждивении
    - желающие купить автомобиль или получить образование
    - с доходом свыше 1 млн